In [1]:
from pathlib import Path
import pandas as pd

dist = Path("E:/OneDrive/Documents/Ontology-Tradecraft/projects/project-3/assignment/dist")

files = list(dist.glob("*.xlsx"))
print("Files:", [f.name for f in files])


Files: ['bfo-core-ies-structural-matches.xlsx', 'ccom-qudt-structural-matches.xlsx', 'ccot-time-structural-matches.xlsx']


In [2]:
# Open the two important files
ccom_qudt = pd.read_excel(dist/"ccom-qudt-structural-matches.xlsx")
ccot_time = pd.read_excel(dist/"ccot-time-structural-matches.xlsx")

print("CCOM-QUDT:", len(ccom_qudt))
display(ccom_qudt.head(10))

print("\nCCOT-TIME:", len(ccot_time))
display(ccot_time.head(10))


CCOM-QUDT: 588


,left_iri,left_label,left_shape,left_axioms,right_iri,right_label,right_shape,right_axioms
0,https://www.commoncoreontologies.org/ont00001206,Multi-Year Temporal Interval,R:some,SubClassOf: obo:BFO_0000038 | SubClassOf: cco:...,http://qudt.org/schema/qudt#FinancialUnit,Financial Unit,R:has,SubClassOf: qudt-v1:ResourceUnit | SubClassOf:...
1,https://www.commoncoreontologies.org/ont00001206,Multi-Year Temporal Interval,R:some,SubClassOf: obo:BFO_0000038 | SubClassOf: cco:...,http://qudt.org/schema/qudt#HumanUnit,Human Unit,R:has,SubClassOf: qudt-v1:ResourceUnit | SubClassOf:...
2,https://www.commoncoreontologies.org/ont00001206,Multi-Year Temporal Interval,R:some,SubClassOf: obo:BFO_0000038 | SubClassOf: cco:...,http://qudt.org/schema/qudt#MechanicsUnit,Mechanics Unit,R:has,SubClassOf: qudt-v1:PhysicalUnit | SubClassOf:...
3,https://www.commoncoreontologies.org/ont00001206,Multi-Year Temporal Interval,R:some,SubClassOf: obo:BFO_0000038 | SubClassOf: cco:...,http://qudt.org/schema/qudt#BiomedicalUnit,Biomedical Unit,R:has,SubClassOf: qudt-v1:ScienceAndEngineeringUnit ...
4,https://www.commoncoreontologies.org/ont00001206,Multi-Year Temporal Interval,R:some,SubClassOf: obo:BFO_0000038 | SubClassOf: cco:...,http://qudt.org/schema/qudt#SpaceAndTimeUnit,Space And Time Unit,R:has,SubClassOf: qudt-v1:PhysicalUnit | SubClassOf:...
5,https://www.commoncoreontologies.org/ont00001206,Multi-Year Temporal Interval,R:some,SubClassOf: obo:BFO_0000038 | SubClassOf: cco:...,http://qudt.org/schema/qudt#PhysicalUnit,Physical Unit,R:has,SubClassOf: qudt-v1:ScienceAndEngineeringUnit ...
6,https://www.commoncoreontologies.org/ont00001206,Multi-Year Temporal Interval,R:some,SubClassOf: obo:BFO_0000038 | SubClassOf: cco:...,http://qudt.org/schema/qudt#ThermodynamicsUnit,Thermodynamics Unit,R:has,SubClassOf: qudt-v1:PhysicalUnit | SubClassOf:...
7,https://www.commoncoreontologies.org/ont00001206,Multi-Year Temporal Interval,R:some,SubClassOf: obo:BFO_0000038 | SubClassOf: cco:...,http://qudt.org/schema/qudt#ScienceAndEngineer...,Science And Engineering Unit,R:has,SubClassOf: qudt-v1:Unit | SubClassOf: qudt-v1...
8,https://www.commoncoreontologies.org/ont00001206,Multi-Year Temporal Interval,R:some,SubClassOf: obo:BFO_0000038 | SubClassOf: cco:...,http://qudt.org/schema/qudt#ChemistryUnit,Chemistry Unit,R:has,SubClassOf: qudt-v1:PhysicalUnit | SubClassOf:...
9,https://www.commoncoreontologies.org/ont00001206,Multi-Year Temporal Interval,R:some,SubClassOf: obo:BFO_0000038 | SubClassOf: cco:...,http://qudt.org/schema/qudt#AtomicPhysicsUnit,Atomic Physics Unit,R:has,SubClassOf: qudt-v1:PhysicalUnit | SubClassOf:...



CCOT-TIME: 64


,left_iri,left_label,left_shape,left_axioms,right_iri,right_label,right_shape,right_axioms
0,https://www.commoncoreontologies.org/ont00000800,Day,R:some,SubClassOf: obo:BFO_0000202 | SubClassOf: cco:...,http://www.w3.org/2006/time#TimePosition,Time position,R:card=1×2,SubClassOf: TemporalPosition | SubClassOf: (nu...
1,https://www.commoncoreontologies.org/ont00000800,Day,R:some,SubClassOf: obo:BFO_0000202 | SubClassOf: cco:...,http://www.w3.org/2006/time#January,January,R:has×2,SubClassOf: DateTimeDescription | SubClassOf: ...
2,https://www.commoncoreontologies.org/ont00000800,Day,R:some,SubClassOf: obo:BFO_0000202 | SubClassOf: cco:...,http://www.w3.org/2006/time#Duration,Time duration,R:card=1×2,SubClassOf: TemporalDuration | SubClassOf: num...
3,https://www.commoncoreontologies.org/ont00000800,Day,R:some,SubClassOf: obo:BFO_0000202 | SubClassOf: cco:...,http://www.w3.org/2006/time#TemporalPosition,Temporal position,R:card=1,SubClassOf: hasTRS exactly 1
4,https://www.commoncoreontologies.org/ont00000329,Multi-Month Temporal Interval,R:some,SubClassOf: obo:BFO_0000038 | SubClassOf: cco:...,http://www.w3.org/2006/time#TimePosition,Time position,R:card=1×2,SubClassOf: TemporalPosition | SubClassOf: (nu...
5,https://www.commoncoreontologies.org/ont00000329,Multi-Month Temporal Interval,R:some,SubClassOf: obo:BFO_0000038 | SubClassOf: cco:...,http://www.w3.org/2006/time#January,January,R:has×2,SubClassOf: DateTimeDescription | SubClassOf: ...
6,https://www.commoncoreontologies.org/ont00000329,Multi-Month Temporal Interval,R:some,SubClassOf: obo:BFO_0000038 | SubClassOf: cco:...,http://www.w3.org/2006/time#Duration,Time duration,R:card=1×2,SubClassOf: TemporalDuration | SubClassOf: num...
7,https://www.commoncoreontologies.org/ont00000329,Multi-Month Temporal Interval,R:some,SubClassOf: obo:BFO_0000038 | SubClassOf: cco:...,http://www.w3.org/2006/time#TemporalPosition,Temporal position,R:card=1,SubClassOf: hasTRS exactly 1
8,https://www.commoncoreontologies.org/ont00001058,Hour,R:some,SubClassOf: obo:BFO_0000202 | SubClassOf: cco:...,http://www.w3.org/2006/time#TimePosition,Time position,R:card=1×2,SubClassOf: TemporalPosition | SubClassOf: (nu...
9,https://www.commoncoreontologies.org/ont00001058,Hour,R:some,SubClassOf: obo:BFO_0000202 | SubClassOf: cco:...,http://www.w3.org/2006/time#January,January,R:has×2,SubClassOf: DateTimeDescription | SubClassOf: ...


In [3]:
# Show just the key columns so it's readable
ccot_time[['label_a','label_b']].head(20)
ccom_qudt[['label_a','label_b']].head(20)


KeyError: "None of [Index(['label_a', 'label_b'], dtype='object')] are in the [columns]"

In [ ]:
import os, glob, pandas as pd

# adjust this if your notebook is elsewhere
DIST = "projects/project-3/assignment/dist"

paths = sorted(glob.glob(os.path.join(DIST, "*.xlsx")))
print("Found", len(paths), "files in", DIST)
for p in paths:
    try:
        df = pd.read_excel(p)
        print(os.path.basename(p), "->", len(df), "rows | cols:", list(df.columns)[:6])
    except Exception as e:
        print(os.path.basename(p), "-> couldn't read:", e)


In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON

BASE = "http://127.0.0.1:5000/sparql"
keys = ["bfo","ies","ccom","qudt","ccot","to"]

def sample_triples(url):
    sp = SPARQLWrapper(url)
    sp.setReturnFormat(JSON)
    sp.setQuery("SELECT ?s ?p ?o WHERE { ?s ?p ?o } LIMIT 5")
    return sp.query().convert()["results"]["bindings"]

for k in keys:
    url = f"{BASE}/{k}"
    try:
        rows = sample_triples(url)
        print(f"{k}: sample triples returned = {len(rows)}")
    except Exception as e:
        print(f"{k}: ERROR -> {e}")


In [ ]:
# Install if needed:
# !pip install SPARQLWrapper pandas openpyxl
!pip install SPARQLWrapper pandas openpyxl

from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd

# Point to your running server
BASE_URL = "http://127.0.0.1:5000/sparql"
BASE = BASE_URL

# Map endpoint key -> output Excel filename (DICT, not a set)
TARGETS = {
    "bfo":  "bfo-class.xlsx",
    "ies":  "ies-class.xlsx",
    "ccom": "ccom-class.xlsx",
    "qudt": "qudt-class.xlsx",
    "ccot": "ccot-class.xlsx",
    "to":   "time-class.xlsx", 
}



CLASS_LABEL_QUERY = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl:  <http://www.w3.org/2002/07/owl#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT ?iri ?label
WHERE {
  {
    ?iri a owl:Class
  } UNION {
    ?iri a rdfs:Class
  }
  OPTIONAL { ?iri rdfs:label ?rdfsLabel }
  OPTIONAL { ?iri skos:prefLabel ?skosLabel }
  BIND(COALESCE(?rdfsLabel, ?skosLabel, STRAFTER(STR(?iri), "#")) AS ?label)
  FILTER NOT EXISTS { ?iri owl:deprecated true }
}
ORDER BY ?label


"""



def run_select(endpoint_url: str, query: str) -> pd.DataFrame:
    sp = SPARQLWrapper(endpoint_url)
    sp.setQuery(query)
    sp.setReturnFormat(JSON)
    data = sp.query().convert()
    rows = []
    for b in data["results"]["bindings"]:
        iri = b.get("iri", {}).get("value")
        label = b.get("label", {}).get("value")
        rows.append({"iri": iri, "label": label})
    return pd.DataFrame(rows, columns=["iri", "label"])

for key, outfile in TARGETS.items():
    endpoint = f"{BASE}/{key}"
    print(f"Querying {endpoint} …")
    df = run_select(endpoint, CLASS_LABEL_QUERY)
    # Save to Excel
    df.to_excel(outfile, index=False)
    print(f"Saved {len(df)} rows to {outfile}")
